In [53]:
class Graph:
    def __init__(self, vertices, edges):
        self.vertices = vertices
        self.edges = edges
        self.graph = {str(vertex): set([]) for vertex in range(-self.vertices, self.vertices + 1)}
        self.reversed_graph = {str(vertex): set([]) for vertex in range(-self.vertices, self.vertices + 1)}
        self.visited = set()
        self.stack = []
        self.scc_groups = []
        self.topological_sort = []

    def add_edge(self, origin, destination):
        implications = self.get_implications(origin, destination)

        for implication in implications:
            self.graph[str(implication[0])].add(str(implication[1]))
            self.reversed_graph[str(implication[1])].add(str(implication[0]))

    def get_implications(self, p, q):
        implication1 = (-p, q)
        implication2 = (-q, p)
        return [implication1, implication2]

    def get_graph(self):
        return self.graph
    
    def get_vertices(self):
        return self.vertices

    def get_scc_groups(self):
        return self.scc_groups

    def first_run(self):
        for vertex in range(-self.vertices, self.vertices + 1):
            vert = str(vertex)
            if vert != "0":
                if vert not in self.visited:
                    self.dfs(vert, self.graph)
        self.visited.clear()

    def get_connected(self):
        count = 0
        while len(self.stack) > 0:
            element = self.stack.pop()
            group = set([])
            if element not in self.visited:
                self.dfs(element, self.reversed_graph, group)
                count += 1
                self.scc_groups.append(group)
        return self.validate_satisfability()

    def validate_satisfability(self):
        answer = []
        for element in self.scc_groups[0]:
            answer.append(element)
        my_dict = {vertex: set() for vertex in answer}
        #print(f'this is my scc groups {self.scc_groups}')

        for element in my_dict:
            if str(-int(element)) in my_dict:
                return "0"

        self.do_topological_sort()
        return self.get_sorted()
        

    def get_sorted(self):
        answer = [0] * (self.vertices + 1)
        answer[0] = "1"

        for index in range (self.vertices):
            element = int(self.topological_sort[index])
            #print(f'this is the element {element} nd this is the abs value {abs(element)}')
            if answer[abs(element)] == 0:
                answer[abs(element)] = str(element)
            if 0 not in answer:
                break
        return answer

    def dfs(self, start, graph, group = None):
        self.visited.add(start)
        if group != None:
            group.add(start)

        for element in graph[start]:
            if element not in self.visited:
                self.dfs(element, graph, group)

        self.stack.append(start)

    def do_topological_sort(self):
        self.visited.clear()
        for group in self.scc_groups:
            for element in group:
                if element not in self.visited:
                    self.visited.add(element)
                    self.dfs_traversal(element, {vertex: [] for vertex in list(group)})
        return self.topological_sort[:self.vertices + 1]
    
    def dfs_traversal(self, node, group):
        for neighbor in group[node]:
            if neighbor not in self.visited:
                self.visited.add(neighbor)
                self.dfs_traversal(neighbor)
        self.topological_sort.append(node)
            

In [54]:
with open("2sat_test.txt", "r") as f:
    graphs = int(f.readline())
    result = []

    for graph in range(graphs):
        f.readline()
        vertices, edges = map(int, f.readline().split())
        g = Graph(vertices, edges)

        for edge in range(edges):
            nodea, nodeb = map(int, f.readline().split())
            g.add_edge(nodea, nodeb)
        
        g.first_run()
        result.append(g.get_connected())

with open("final_result.txt", "w") as f:
    for group in result:
        if type(group) == "int":
            f.write(group)
        else:
            for member in group:
                f.write(member)
                f.write(" ")
        f.write("\n")